## TensorNetwork: A Library for Physics and Machine Learning

#### 1. Introduction
张量网络是为高效表示和处理高维数据而设计的稀疏数据结构。被广泛的应用与凝聚态物理，量子化学，统计力学，量子理论，甚至量子引力和宇宙学。

现在有人将张量网络应用于机器学习中，Stoudenmire和Schwab将MPS应用与MINST数据集的分类问题中。Levine将ConvAC网络与树形张量网络等价起来。在张量网络和卷积网络结构之间构建了很广泛的联系。
Section2用来复习张量网络的一些基础知识，Section3使用tensornetwork这个API实现一些常见的张量操作，TN这个库主要是基于tensorflow搭建的。

#### 2. An Overview of Tensor Network
从图论的角度来说，一个order为r的张量可以表示为一个node，这个node的degree为r，或者我们称之为“leg”。所有的lines都表示一个index。

下面介绍张量的缩并(contraction)，假设我们给定两个向量$A$和$B$，这两个向量维度相同，我们做内积运算：$A\cdot B=\sum_{i}A_i B_i$.如果用符号来表示我们可以将$A$和$B$的leg连接起来，当我们这样做了以后，我们会发现就没有free的leg了，这就反映了对于$A\cdot B$不存在剩余的index。换句话说，$A\cdot B$是一个标量。类似地，我们会发现，矩阵和向量相乘就是一个向量，在图中表示就是存在一个index。
![Examples of graphical representation of tensors](tensor_graphical.png)

除了这些基本的例子，一个有用的张量网络是通过把几个节点连接在一起作为一个更大的结构的一部分来实现的。其结果是由几个较小的张量的收缩而成的具有许多index的总网络。网络本身可以看作是一个非常高维的数组，但它是一个非常稀疏的数组。张量网络是稀疏数据的有效表示，通过对张量网络的智能操作，可以在不需要巨大代价的情况下有效地处理高维数据。

#### 3. The TensorNetwork API

In [4]:
#这一部分主要熟悉一下tensornetwork这个库的一些内置函数
import numpy as np
import tensorflow as tf
import tensornetwork as tn

A `TensorNetwork` is the main object of the library. It keeps track of its own set of `Node` objects, and contains methods to add additional nodes, connect them with edges, contract them, and manipulate them in other ways.

In [6]:
#首先熟悉node的概念，这是搭建tn的基础，在计算中node表示的是一个tensor。
#每个axis有对应的一条边，这些边可以连接别的node，edges的数量表示这个tensor的order。
node = tn.Node(np.eye(2))
print(node.tensor)

[[1. 0.]
 [0. 1.]]


In [12]:
a = tn.Node(np.eye(2))
b = tn.Node(np.eye(2))
c = tn.Node(np.eye(2))
#当创建一个node的时候会自动产生悬挂着的edge
dangling_edge = a.get_edge(1)
#或者使用dangling_edge = a[1]
#创建一条新的边通过将两个dangling的连接起来
standard_edge = a[0] ^ b[0]
#也可以使用standard_edge = tn.connect(a[0],b[0])
#连接节点自身形成自环创建一个trace edge
trace_edge = c[0] ^ c[1]
print(tn.contract(trace_edge).tensor)

2.0


In [10]:
d = tn.Node(np.ones(2))
e = tn.Node(np.ones(2))
edge_de = d[0] ^ e[0]
f = tn.contract(edge_de)
print(f.tensor)

2.0
